In [1]:
import pandas as pd
import numpy as np

In [2]:
datos = pd.read_csv("http://www.mambiente.munimadrid.es/opendata/horario.txt", header=None, dtype="str")
datos.head()

,0,1,2,3,4,5,6,7,8,9,...,47,48,49,50,51,52,53,54,55,56
0,28,079,004,01,38,02,2018,10,04,00012,...,00000,N,00000,N,00000,N,00000,N,00000,N
1,28,079,004,06,48,02,2018,10,04,000.7,...,00000,N,00000,N,00000,N,00000,N,00000,N
2,28,079,004,07,08,02,2018,10,04,00073,...,00000,N,00000,N,00000,N,00000,N,00000,N
3,28,079,004,08,08,02,2018,10,04,00087,...,00000,N,00000,N,00000,N,00000,N,00000,N
4,28,079,004,12,08,02,2018,10,04,00199,...,00000,N,00000,N,00000,N,00000,N,00000,N


In [3]:
col_horas = ["{}_{}".format(h, tipo) for h in range(1, 25) for tipo in ("medida", "valido")]

datos = pd.read_csv("http://www.mambiente.munimadrid.es/opendata/horario.txt",
                    dtype="str",
                    names=["estacion_parte1", "estacion_parte2", "estacion_parte3", 
                           "parametro", "tecnica", "periodo", "yyyy", "mm", "dd"] + col_horas)
datos.head()

,estacion_parte1,estacion_parte2,estacion_parte3,parametro,tecnica,periodo,yyyy,mm,dd,1_medida,...,20_medida,20_valido,21_medida,21_valido,22_medida,22_valido,23_medida,23_valido,24_medida,24_valido
0,28,079,004,01,38,02,2018,10,04,00012,...,00000,N,00000,N,00000,N,00000,N,00000,N
1,28,079,004,06,48,02,2018,10,04,000.7,...,00000,N,00000,N,00000,N,00000,N,00000,N
2,28,079,004,07,08,02,2018,10,04,00073,...,00000,N,00000,N,00000,N,00000,N,00000,N
3,28,079,004,08,08,02,2018,10,04,00087,...,00000,N,00000,N,00000,N,00000,N,00000,N
4,28,079,004,12,08,02,2018,10,04,00199,...,00000,N,00000,N,00000,N,00000,N,00000,N


In [4]:
datos["estacion"] = datos.estacion_parte1 + datos.estacion_parte2 + datos.estacion_parte3
datos["fecha"] = datos.yyyy + datos.mm + datos.dd
 
datos = datos[["estacion", "fecha", "parametro"] + col_horas]
datos.head()

,estacion,fecha,parametro,1_medida,1_valido,2_medida,2_valido,3_medida,3_valido,4_medida,...,20_medida,20_valido,21_medida,21_valido,22_medida,22_valido,23_medida,23_valido,24_medida,24_valido
0,28079004,20181004,01,00012,V,00012,V,00011,V,00010,...,00000,N,00000,N,00000,N,00000,N,00000,N
1,28079004,20181004,06,000.7,V,000.5,V,000.4,V,000.4,...,00000,N,00000,N,00000,N,00000,N,00000,N
2,28079004,20181004,07,00073,V,00060,V,00031,V,00025,...,00000,N,00000,N,00000,N,00000,N,00000,N
3,28079004,20181004,08,00087,V,00080,V,00069,V,00061,...,00000,N,00000,N,00000,N,00000,N,00000,N
4,28079004,20181004,12,00199,V,00173,V,00116,V,00099,...,00000,N,00000,N,00000,N,00000,N,00000,N


In [5]:
datos = datos.melt(id_vars=["estacion", "fecha", "parametro"])
datos.head()

,estacion,fecha,parametro,variable,value
0,28079004,20181004,01,1_medida,00012
1,28079004,20181004,06,1_medida,000.7
2,28079004,20181004,07,1_medida,00073
3,28079004,20181004,08,1_medida,00087
4,28079004,20181004,12,1_medida,00199


In [6]:
datos[["hora", "variable"]] = datos.variable.str.split("_", expand=True)
datos.head()

,estacion,fecha,parametro,variable,value,hora
0,28079004,20181004,01,medida,00012,1
1,28079004,20181004,06,medida,000.7,1
2,28079004,20181004,07,medida,00073,1
3,28079004,20181004,08,medida,00087,1
4,28079004,20181004,12,medida,00199,1


In [7]:
datos = pd.pivot_table(datos, 
                        index=["estacion", "fecha", "parametro", "hora"], 
                        columns="variable", values="value", aggfunc='first').reset_index()

In [8]:
datos.head()

variable,estacion,fecha,parametro,hora,medida,valido
0,28079004,20181004,01,1,00012,V
1,28079004,20181004,01,10,00021,V
2,28079004,20181004,01,11,00018,V
3,28079004,20181004,01,12,00012,V
4,28079004,20181004,01,13,00011,V


In [9]:
datos["datetime"] = pd.to_datetime(datos.fecha) + datos.hora.astype('timedelta64[h]')
datos["contaminante"] = datos.parametro.astype('category').cat.rename_categories({
    '01':'SO2', '06':'CO', '07':'NO', '08':'NO2', '09':'PM2.5', 
    '10':'PM10', '12':'NOx', '14':'O3', '20':'TOL', '30':'BEN', 
    '35':'EBE', '42':'TCH', '43':'CH4', '44':'NMHC'
})
datos["medida"] = datos.medida.astype('float')
datos["valido"] = datos.valido == 'V'

datos = datos[["estacion", "datetime", "contaminante", "medida", "valido"]]

In [10]:
datos.head()

variable,estacion,datetime,contaminante,medida,valido
0,28079004,2018-10-04 01:00:00,SO2,12.0,True
1,28079004,2018-10-04 10:00:00,SO2,21.0,True
2,28079004,2018-10-04 11:00:00,SO2,18.0,True
3,28079004,2018-10-04 12:00:00,SO2,12.0,True
4,28079004,2018-10-04 13:00:00,SO2,11.0,True
